# IIW 2021 joint intermediate meeting C-I, C-IV, C-XII, SG-212
This notebook is an extended interactive version of the `weldx` features and example dataset presented at the **2021 IIW joint intermediate meeting of C-I, C-IV, C-XII & SG-212**.

The code of this notebook can be found here: https://github.com/BAMWelDX/IIW2021_joint_intermediate_CXII

You can launch this notebook as an interactive binder session in your browser following this link: 
https://mybinder.org/v2/gh/BAMWelDX/IIW_2021_joint_intermediate_CXII/main?urlpath=lab/tree/iiw2021_CXII_fabry_cagtay_01.ipynb

The `weldx` documentation and code is available online:\
https://weldx.readthedocs.io/en/latest/ \
https://github.com/BAMWelDX/weldx

The `weldx` documentation and GitHub links for this specific code version `v0.3.1` can be found here:\
https://weldx.readthedocs.io/en/v0.3.1/index.html \
https://github.com/BAMWelDX/weldx/tree/v0.3.1



## Process Video
To give an ovierview of this welding example, here is a video recording of the welding experiment conducted at BAM.

We can see the pre- and post-welding scan of the workpiece geometry as well as the position of the temperature measurements.

<video controls src="./data/WID417.webm" />

## Imports
We start with some general python package imports used throughout this notebook.

In [ ]:
import pprint
from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd

import asdf
import weldx
from weldx import Q_
from weldx.asdf.util import asdf_json_repr

pp = pprint.PrettyPrinter(indent=2)
pprint = pp.pprint

Some helper functions for this notebook are included in the `helpers.py` file.

In [ ]:
from helpers import (
    add_axis_labels_3d,
    ax_setup,
    build_base_csm,
    create_geometry,
    cs_colors,
    plot_gmaw,
    plot_measurements,
    plot_signal,
    welding_wire_geo_data,
    ipympl_style
)

## opening the file
To open and access the file contents we will use the `asdf` library to open the weldx example dataset.

We define the ASDF filename that contains the data used for this notebook.

In [ ]:
filename = "./data/single_pass_weld.asdf"

To get an overview of the file contents we can use the `asdf_json_repr` function of the weldx library to create an interactive tree view.

Try searching for a specific term like `wire_feedrate` using the ***Filter...*** box in the upper right.

In [ ]:
asdf_json_repr(filename)

The file is expected to validate against the schema `single_pass_weld-1.0.0.schema.yaml`.
The details describing the schema requirements can be found here: https://weldx.readthedocs.io/en/v0.3.1/generated/weldx.bam.de/weldx/datamodels/single_pass_weld-1.0.0.schema.html

In [ ]:
file_schema = (
    Path(weldx.__path__[0])
    / Path(
        "./asdf/schemas/weldx.bam.de/weldx/datamodels/single_pass_weld-1.0.0.schema.yaml"
    ).as_posix()
)

We open the weldx file and run a validation agains the `single_pass_weld-1.0.0.schema.yaml` schema.\
The validation ensures that all elements of the weldx file pass the requirements defined in the schema, including:
- all requirement entries are present in the file:
  - `workpiece`
  - `TCP`
  - `welding_current`
  - `welding_voltage`
  - `measurement chains`
  - `equipment`
- all entries and objects stored in the file have the correct type
- all additional restrictions defined in `single_pass_weld-1.0.0.schema.yaml` are met

In [ ]:
with asdf.open(filename, custom_schema=file_schema) as asdf_file:
    weldx_file = asdf_file.tree

## general metadata
First let's look at some general simple metadata stored in the WelDX-file.

The `reference_timestamp` field is used to indicate the start time of the experiment (the moment of arc ignition). All time data that is not given as absolute time are interpreted as relative to the given reference time.

In [ ]:
weldx_file.get("reference_timestamp")

We can deduce the total runtime of the experiment from the TCP movement of the welding.

In [ ]:
pd.TimedeltaIndex(weldx_file["TCP"].time)

In [ ]:
weldx_file.get("wx_user")

In [ ]:
t = pd.TimedeltaIndex(weldx_file["TCP"].time[[0, -1]])

## workpiece definition
The file schema mandates that the user provides workpiece information with the following properties:
- `base_metal` referenced by a common name and the associated standard
- the `geometry` consisting of a groove description following ISO 9692-1 and the seam length

Here is how this information is stored the `workpiece` entry of the example dataset weldx file:
```yaml
workpiece:
  base_metal: {common_name: 'S355J2+N', standard: 'DIN EN 10225-2:2011'}
  geometry:
    groove_shape: !<tag:weldx.bam.de:weldx/groove/iso_9692_1_2013_12/VGroove-1.0.0>
      t: !unit/quantity-1.1.0 {unit: millimeter, value: 8}
      alpha: !unit/quantity-1.1.0 {unit: degree, value: 45}
      b: !unit/quantity-1.1.0 {unit: millimeter, value: 1}
      c: !unit/quantity-1.1.0 {unit: millimeter, value: 1}
    seam_length: !unit/quantity-1.1.0 {unit: millimeter, value: 350}
```

### workpiece material

Since we know exactly where to find the information in the file, we can access the metadata directly for all files that validate against the file schema.

In [ ]:
weldx_file["workpiece"]["base_metal"]["common_name"]

In [ ]:
weldx_file["workpiece"]["base_metal"]["standard"]

### seam length

The total seam length of the workpiece is also stored.\
As throughout most of the functionality of the `weldx` API, physical units must be used where appropriate to avoid ambiguity.

In [ ]:
seam_length = weldx_file["workpiece"]["geometry"]["seam_length"]
print(seam_length)

### welding groove

The groove shape will be loaded into a specific weldx type:

In [ ]:
groove = weldx_file["workpiece"]["geometry"]["groove_shape"]
str(groove)

The `weldx` API includes convinient functions to create and visualize different welding groove shapes.\
Many examples and details are available in this tutorial: https://weldx.readthedocs.io/en/v0.3.1/tutorials/groove_types_01.html

To get a picture of the groove shape we can simply call the `plot` function.

In [ ]:
groove.plot()
fig = plt.gcf(); fig.set_size_inches(7, 7);

### 3D Geometry
With all the metadata of the workpiece available, it is easy to visualize a simple 3D model of the specimen.

In [ ]:
geometry = create_geometry(groove, seam_length, Q_(10, "mm"))
geometry.plot(profile_raster_width=Q_(4, "mm"), trace_raster_width=Q_(60, "mm"))
ax_setup(plt.gca())
fig = plt.gcf()
fig.set_size_inches(7, 7)

## Welding TCP movement description

```yaml
TCP: !<tag:weldx.bam.de:weldx/core/transformations/local_coordinate_system-1.0.0>
  reference_time: !<tag:weldx.bam.de:weldx/time/timestamp-1.0.0> {value: '2021-03-17T11:06:42.334400'}
  time: !<tag:weldx.bam.de:weldx/time/timedeltaindex-1.0.0>
    values: !core/ndarray-1.0.0
      data: [0, 41333333333]
      datatype: int64
      shape: [2]
    start: !<tag:weldx.bam.de:weldx/time/timedelta-1.0.0> {value: 'P0DT0H0M0S'}
    end: !<tag:weldx.bam.de:weldx/time/timedelta-1.0.0> {value: 'P0DT0H0M41.333333333S'}
  coordinates: !<tag:weldx.bam.de:weldx/core/variable-1.0.0>
    name: coordinates
    dimensions: [time, c]
    dtype: <f8
    data: !core/ndarray-1.0.0
      data:
      - [20.0, 0.0, 3.0]
      - [330.0, 0.0, 3.0]
      datatype: float64
      shape: [2, 3]
```

We can add the movement of the welding TCP to the geometry plot:

In [ ]:
csm_base = build_base_csm(weldx_file, plot=False)
csm_base.plot(
    reference_system="workpiece", coordinate_systems=["TCP weld"],
    data_sets=["workpiece (simple)"], colors=cs_colors,
    show_wireframe=True, show_data_labels=False, show_vectors=False,
)
ax_setup(plt.gca())

## Process description
The arc welding process must be defined using the following properties:
```yaml
process:
  type: object
  properties:
    welding_process:
      $ref: "http://weldx.bam.de/schemas/weldx/process/GMAW-1.0.0"
    shielding_gas:
      tag: "tag:weldx.bam.de:weldx/aws/process/shielding_gas_for_procedure-1.0.0"
    weld_speed:
      ...
    welding_wire:
      ...
  required: [welding_process, shielding_gas, weld_speed, welding_wire]
```

In [ ]:
process = weldx_file["process"]
process["welding_process"].tag

The weld speed is restricted to a constant value of dimension "[length]/[time]"
```yaml
weld_speed:
  tag: "tag:weldx.bam.de:weldx/core/time_series-1.0.0"
  wx_unit: "m/s"
  wx_shape: [1]
```

In [ ]:
process["weld_speed"]

In [ ]:
process["welding_wire"]

In [ ]:
pprint(process["shielding_gas"].torch_shielding_gas.__dict__)

In [ ]:
# switch to static plots
%matplotlib inline

In [ ]:
gmaw_process = process["welding_process"]
fig, ax = plot_gmaw(gmaw_process, t)  

## measurements
list all measurements stored in the file

In [ ]:
measurement_data = weldx_file["measurements"]
for measurement in measurement_data:
    print(measurement.name)

In [ ]:
plot_measurements(measurement_data, ref_time=weldx_file["reference_timestamp"])

In [ ]:
# switch to interactive plots
%matplotlib widget

Plot the current meausurement:

In [ ]:
plot_signal(weldx_file["welding_current"])

Plot the voltage measurement:

In [ ]:
plot_signal(weldx_file["welding_voltage"])

Plot a section of the current measurement:

In [ ]:
plot_signal(weldx_file["welding_current"], limits=(23, 23.025))

## measurement chains
To document how a welding related measurement was conducted, we can describe and store measurement chains using the `weldx` API.
This includes:
- describing the measurement equipment
- describing multiple transformation steps from raw-data to the final measurement
- providing information about measurement uncertainties and errors
- attaching certification examples or similar files

An in depth example describing measurement chains can be found in the documentation: [meassurement_example](https://weldx.readthedocs.io/en/v0.3.1/tutorials/measurement_example.html)

### current measurement chain

In [ ]:
current_measurement_chain = measurement_data[0].measurement_chain

current_source = current_measurement_chain.data_source
print(current_source.name)
for processor in current_measurement_chain.data_processors:
    print(processor.name)

In [ ]:
fig, ax = plt.subplots(nrows=1, figsize=(12, 6))
weldx_file["measurements"][0].measurement_chain.plot(ax)
ipympl_style(fig,toolbar=False)

In [ ]:
fig, ax = plt.subplots(nrows=1, figsize=(12, 6))
weldx_file["measurements"][1].measurement_chain.plot(ax)
ipympl_style(fig,toolbar=False)

In [ ]:
fig, ax = plt.subplots(nrows=1, figsize=(12, 6))
weldx_file["measurements"][2].measurement_chain.plot(ax)
ipympl_style(fig,toolbar=False)

## coordinate systems
The `weldx` API contains multiple functions to describe dependencies and transformations between multiple different coordinate systems.

- translations and rotations
- constant and time dependent transformations
- transformation between different systems
- grouping multiple systems into subsystems
- transforming spatial data between different coordinate systems
- visualization of transformations and systems

There are multiple tutorials available covering coordinate transformations using the `LocalCoordinateSystem` and `CoordinateSystemManager` classes:

- https://weldx.readthedocs.io/en/v0.3.1/tutorials/transformations_01_coordinate_systems.html
- https://weldx.readthedocs.io/en/v0.3.1/tutorials/transformations_02_coordinate_system_manager.html
- https://weldx.readthedocs.io/en/v0.3.1/tutorials/transformations_02_coordinate_system_manager.html#Visualizing-the-coordinate-systems-of-the-CSM
- https://weldx.readthedocs.io/en/v0.3.1/tutorials/welding_example_02_weaving.html

In [ ]:
# switch to static plots
%matplotlib inline

In addition to the simplified weldment specification, the example dataset contains the complete coordinate system information describing the BAM arc welding setup.

- the definition of the reference user frame used for robot programming
- the recorded actual TCP movement of the robot
- the movement of a laser line scanner attached to the robot head

We can load the instance of the coordinate system manager directly from the weldx file.\
Following the file schema the data can be accessed under the key `coordinate_systems`.

In [ ]:
csm = weldx_file["coordinate_systems"]

We can visualize all loaded coordinate systems using the built-in `plot` functions.

In [ ]:
csm.plot_graph()
plt.gcf().set_size_inches(w=6, h=6)

Let's take another look at the weld specimen.

<img src='./data/WID417_pre.JPG' width="640" />

The workpiece coordinate system has it's origin located at the start of the workpiece at groove center. We can calculate the position of the thermocouple placement in the workpiece coordinate system.

In [ ]:
csm.get_cs("T1","user_frame")

The second thermocouple is offset by 5 mm from the first.

In [ ]:
csm.get_cs("TCP","workpiece")

### Add geometry data to CSM

For advanced visualization using the `k3d` backend we attach some 3D data to the `CoordinateSystemManager`

In [ ]:
geometry_full_width = create_geometry(groove, seam_length, Q_(100, "mm"))
spatial_data_geo_full = geometry_full_width.spatial_data(
    profile_raster_width=Q_(4, "mm"), trace_raster_width=Q_(60, "mm")
)
spatial_data_geo_full.coordinates = spatial_data_geo_full.coordinates.astype("float32")

spatial_data_geo_reduced = geometry.spatial_data(
    profile_raster_width=Q_(4, "mm"), trace_raster_width=Q_(60, "mm")
)

csm.assign_data(spatial_data_geo_full, "workpiece geometry", "workpiece")
csm.assign_data(spatial_data_geo_reduced, "workpiece geometry (reduced)", "workpiece")

Adding 3D measurements of the actual pre- and post-weld workpiece geometry.

In [ ]:
with asdf.open(f"./data/scans.asdf") as af:
    scans = af.tree
    csm.assign_data(scans["scan_0"], "scan_0", "workpiece")
    csm.assign_data(scans["scan_1"], "scan_1", "workpiece")

We also add a 3D model of the welding wire.

In [ ]:
welding_wire_diameter = weldx_file["process"]["welding_wire"]["diameter"].m
csm.assign_data(
    welding_wire_geo_data(welding_wire_diameter / 2, 17, 16), "welding_wire", "TCP"
)

The `weldx` API offers some advanced 3D visualization that run inside the browser window.

Here is an example visualization of the experiment design.\
The reconstruction is entierly based on the metadata stored inside the weldx file.

In [ ]:
csm.plot(
    reference_system="workpiece", coordinate_systems=["TCP design", "T1", "T2"],
    data_sets=["workpiece geometry", "welding_wire"], colors=cs_colors,
    show_data_labels=True, backend="k3d",
)

In [ ]:
csm.plot(
    reference_system="workpiece", coordinate_systems=["TCP", "T1", "T2"],
    data_sets=["scan_0", "welding_wire"], colors=cs_colors,
    show_data_labels=True, backend="k3d",
)

## A small example

What was the position of the welding process at a specific time (disturbance) of the experiment?

In [ ]:
csm

In [ ]:
csm.get_cs("TCP","workpiece",time=Q_(30,"s"))

## Equipment

In [ ]:
asdf_json_repr(filename, ("equipment",))

### Simple Error estimation

In [ ]:
current_source = current_measurement_chain.data_source
print(current_source.error)
for processor in current_measurement_chain.data_processors:
    print(processor.error)

In [ ]:
total_error = current_source.error.deviation + 1
for processor in current_measurement_chain.data_processors:
    total_error *= processor.error.deviation + 1
total_error -= 1
total_error

In [ ]:
welding_current = weldx_file["welding_current"]
welding_current_cycle_peak = welding_current.data.data[9999:10017]
time = welding_current.data.data.time[9999:10017] - welding_current.data.data.time[0]
time = weldx.util.pandas_time_delta_to_quantity(time)
plt.plot(time.m, welding_current_cycle_peak, color="b")
plt.plot(
    time.m, welding_current_cycle_peak * (1 + total_error).magnitude, "--", color="b"
)
plt.plot(
    time.m, welding_current_cycle_peak * (1 - total_error).magnitude, "--", color="b"
)
plt.gca().set_xlabel("time in s")
plt.gca().set_ylabel("Welding current in A")

### Show subsystems

In [ ]:
csm.subsystem_names

In [ ]:
import matplotlib.pyplot as plt

subsystems = csm.subsystems

fig, ax = plt.subplots(ncols=2)
fig.set_size_inches(w=14, h=5)
for i, subsystem in enumerate(subsystems):
    subsystem.plot_graph(ax=ax[i])

Here is a simple matplotlib plot showing the position of the thermocouples on the workpiece surface

In [ ]:
csm.plot(
    reference_system="workpiece", coordinate_systems=["TCP design", "T1", "T2"],
    data_sets=["workpiece geometry (reduced)"], colors=cs_colors,
    show_wireframe=True, show_data_labels=False, show_vectors=False,
)
ax_setup(plt.gca())